In [10]:
from bs4 import BeautifulSoup
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

In [2]:

def kml_to_df_simple(kml_file):
    with open(kml_file, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'xml')
    
    data = []
    
    for pm in soup.find_all('Placemark'):
        row = {'Nombre': pm.find('name').text if pm.find('name') else ''}
        
        # Procesar descripción
        desc = pm.find('description')
        if desc:
            desc_soup = BeautifulSoup(desc.text, 'html.parser')
            table = desc_soup.find('table')
            if table:
                for tr in table.find_all('tr'):
                    tds = tr.find_all('td')
                    if len(tds) >= 2:
                        key = tds[0].get_text(strip=True).replace(':', '')
                        value = tds[1].get_text(strip=True)
                        row[key] = value
        
        # Procesar coordenadas
        point = pm.find('Point')
        if point and point.find('coordinates'):
            coords = point.coordinates.text.split(',')
            row['Longitud'] = coords[0].strip()
            row['Latitud'] = coords[1].strip()
        
        data.append(row)
    
    return pd.DataFrame(data)

In [15]:
df_presas = kml_to_df_simple("PRESAS_varias.kml")

In [16]:
df_presas.head()

,Nombre,Id,Nom. Oficial,Nom. Común,Estado,Municipio,Latitud (Grados),Longitud (Grados),Corriente,Alt. cortina (m),Mat. cortina,Tipo,Cap. al NAMO (hm3),Año Terminación,Uso del Agua,Organismo Responsable,,Longitud,Latitud
0,50 ANIVERSARIO,1,50 ANIVERSARIO,SAN JOSE DE GRACIA +,Aguascalientes,San José de Gracia,22.1887611111,-102.4653055556,R Blanco,39.20,Mamposteria,Rigidade Gravedad,5.2000,1977,Riego Agua Potable Abrevadero,UR Unidad de Riego,USUARIOS,-102.4653055556,22.1887611111
1,GRAL ABELARDO RODRIGUEZ,2,GRAL ABELARDO RODRIGUEZ,EL ROSARIO +,Aguascalientes,Jesús María,21.9170888889,-102.4276805556,A Morcinique,25.00,Mamposteria,Rigidade Gravedad,15.9960,1934,Riego Abrevadero,UR Unidad de Riego,NaN,-102.4276805556,21.9170888889
2,LAS ADJUNTAS,3,LAS ADJUNTAS,TANQUE LA PURISIMA <,Aguascalientes,Asientos,22.0430500000,-101.9188805556,Canal y A. Nivel de Tinajuela,5.25,Homogenea con proteccion de enrocamiento,Flexiblede Tierra y/o Enrocamiento,0.5080,1800,Riego Abrevadero,Asoc. Usuar. Asociación de Usuarios,Angel Moreno Montoya,-101.9188805556,22.0430500000
3,ALTAR DE YEDRA,4,ALTAR DE YEDRA,,Aguascalientes,Calvillo,21.8041666667,-102.6883333333,A. ADOBES,35.02,Mamposteria,Rigidade Contrafuertes,0.6700,1940,Riego,Asoc. Usuar. Asociación de Usuarios,USUARIOS,-102.6883333333,21.8041666667
4,LOS ADOBES,5,LOS ADOBES,ALTAR DE YEDRA (PRESA CHICA),Aguascalientes,Calvillo,21.8046666667,-102.6933194444,A. ADOBES,15.04,Mamposteria,Rigidade Contrafuertes,0.0400,1900,Riego,Asoc. Usuar. Asociación de Usuarios,ASOCIACION DE ADOBES,-102.6933194444,21.8046666667


In [17]:
df_presas.shape

(6570, 19)

In [18]:
print(list(df_presas))

['Nombre', 'Id', 'Nom. Oficial', 'Nom. Común', 'Estado', 'Municipio', 'Latitud (Grados)', 'Longitud (Grados)', 'Corriente', 'Alt. cortina (m)', 'Mat. cortina', 'Tipo', 'Cap. al NAMO (hm3)', 'Año Terminación', 'Uso del Agua', 'Organismo Responsable', '', 'Longitud', 'Latitud']


In [20]:
# Asegurarse que las coordenadas son numéricas
df_presas['Longitud'] = pd.to_numeric(df_presas['Longitud'])
df_presas['Latitud'] = pd.to_numeric(df_presas['Latitud'])

In [21]:
# Crear geometrías de punto
geometry = [Point(xy) for xy in zip(df_presas['Longitud'], df_presas['Latitud'])]

In [22]:
# Crear el GeoDataFrame
gdf_presas = gpd.GeoDataFrame(df_presas, geometry=geometry)

In [23]:
# Establecer el sistema de referencia espacial (CRS)
# WGS84 (EPSG:4326) es el estándar para coordenadas en grados decimales
gdf = gdf_presas.set_crs('EPSG:4326')

In [25]:
# Verificar el resultado
print(gdf.shape)
print(gdf.crs)  # Debería mostrar 'EPSG:4326'
gdf.head()

(6570, 20)
EPSG:4326


,Nombre,Id,Nom. Oficial,Nom. Común,Estado,Municipio,Latitud (Grados),Longitud (Grados),Corriente,Alt. cortina (m),Mat. cortina,Tipo,Cap. al NAMO (hm3),Año Terminación,Uso del Agua,Organismo Responsable,,Longitud,Latitud,geometry
0,50 ANIVERSARIO,1,50 ANIVERSARIO,SAN JOSE DE GRACIA +,Aguascalientes,San José de Gracia,22.1887611111,-102.4653055556,R Blanco,39.20,Mamposteria,Rigidade Gravedad,5.2000,1977,Riego Agua Potable Abrevadero,UR Unidad de Riego,USUARIOS,-102.465306,22.188761,POINT (-102.46531 22.18876)
1,GRAL ABELARDO RODRIGUEZ,2,GRAL ABELARDO RODRIGUEZ,EL ROSARIO +,Aguascalientes,Jesús María,21.9170888889,-102.4276805556,A Morcinique,25.00,Mamposteria,Rigidade Gravedad,15.9960,1934,Riego Abrevadero,UR Unidad de Riego,NaN,-102.427681,21.917089,POINT (-102.42768 21.91709)
2,LAS ADJUNTAS,3,LAS ADJUNTAS,TANQUE LA PURISIMA <,Aguascalientes,Asientos,22.0430500000,-101.9188805556,Canal y A. Nivel de Tinajuela,5.25,Homogenea con proteccion de enrocamiento,Flexiblede Tierra y/o Enrocamiento,0.5080,1800,Riego Abrevadero,Asoc. Usuar. Asociación de Usuarios,Angel Moreno Montoya,-101.918881,22.043050,POINT (-101.91888 22.04305)
3,ALTAR DE YEDRA,4,ALTAR DE YEDRA,,Aguascalientes,Calvillo,21.8041666667,-102.6883333333,A. ADOBES,35.02,Mamposteria,Rigidade Contrafuertes,0.6700,1940,Riego,Asoc. Usuar. Asociación de Usuarios,USUARIOS,-102.688333,21.804167,POINT (-102.68833 21.80417)
4,LOS ADOBES,5,LOS ADOBES,ALTAR DE YEDRA (PRESA CHICA),Aguascalientes,Calvillo,21.8046666667,-102.6933194444,A. ADOBES,15.04,Mamposteria,Rigidade Contrafuertes,0.0400,1900,Riego,Asoc. Usuar. Asociación de Usuarios,ASOCIACION DE ADOBES,-102.693319,21.804667,POINT (-102.69332 21.80467)


In [26]:
gdf.to_file('inventario_presas.gpkg', driver='GPKG', layer='name')